In [27]:
import urllib.request, json 
#lets get the most up to date data
with urllib.request.urlopen("https://data.cdc.gov/resource/r8kw-7aab.json") as url:
    newIncomingData = json.loads(url.read().decode())
#after this, if we want to clean the data on here first, we can write some code below
#
#example of what we an look at:
#{'data_as_of': '2020-10-19T00:00:00.000',
#'start_week': '2020-02-01T00:00:00.000',
#'end_week': '2020-02-01T00:00:00.000',
#'group': 'By week',
#'state': 'United States',
#'indicator': 'Week-ending',z
#'covid_deaths': '0',
#'total_deaths': '58616',
#'percent_of_expected_deaths': '0.99',
#'pneumonia_deaths': '3799',
#'pneumonia_and_covid_deaths': '0',
#'influenza_deaths': '479',
#'pneumonia_influenza_or_covid_19_deaths': '4278'}
    i = 1
    for line in newIncomingData:
        if 'covid_deaths' in line: #some days it doesn't show covid data,
            #so I put in logic so it doesn't crash
            print("Day: ", line['data_as_of'], " Deaths: ", line['covid_deaths'])
        else:
            print('for today, it doesnt show covid deaths')
        i = i+1
#
#Now we output the data as we want it, no need to worry about merging because we can overwrite old file
with open('data.json', 'w') as outfile:
    json.dump(newIncomingData, outfile)
    print('data has been outputted')

Day:  2020-10-19T00:00:00.000  Deaths:  0
Day:  2020-10-19T00:00:00.000  Deaths:  1
Day:  2020-10-19T00:00:00.000  Deaths:  0
Day:  2020-10-19T00:00:00.000  Deaths:  5
Day:  2020-10-19T00:00:00.000  Deaths:  10
Day:  2020-10-19T00:00:00.000  Deaths:  35
Day:  2020-10-19T00:00:00.000  Deaths:  55
Day:  2020-10-19T00:00:00.000  Deaths:  572
Day:  2020-10-19T00:00:00.000  Deaths:  3165
Day:  2020-10-19T00:00:00.000  Deaths:  10025
Day:  2020-10-19T00:00:00.000  Deaths:  16190
Day:  2020-10-19T00:00:00.000  Deaths:  17075
Day:  2020-10-19T00:00:00.000  Deaths:  15438
Day:  2020-10-19T00:00:00.000  Deaths:  13153
Day:  2020-10-19T00:00:00.000  Deaths:  11173
Day:  2020-10-19T00:00:00.000  Deaths:  9170
Day:  2020-10-19T00:00:00.000  Deaths:  7181
Day:  2020-10-19T00:00:00.000  Deaths:  6115
Day:  2020-10-19T00:00:00.000  Deaths:  4991
Day:  2020-10-19T00:00:00.000  Deaths:  4200
Day:  2020-10-19T00:00:00.000  Deaths:  3802
Day:  2020-10-19T00:00:00.000  Deaths:  3783
Day:  2020-10-19T00:00:

Day:  2020-10-19T00:00:00.000  Deaths:  67
Day:  2020-10-19T00:00:00.000  Deaths:  54
Day:  2020-10-19T00:00:00.000  Deaths:  58
Day:  2020-10-19T00:00:00.000  Deaths:  26
Day:  2020-10-19T00:00:00.000  Deaths:  18
Day:  2020-10-19T00:00:00.000  Deaths:  0
Day:  2020-10-19T00:00:00.000  Deaths:  0
Day:  2020-10-19T00:00:00.000  Deaths:  0
Day:  2020-10-19T00:00:00.000  Deaths:  0
for today, it doesnt show covid deaths
for today, it doesnt show covid deaths
for today, it doesnt show covid deaths
Day:  2020-10-19T00:00:00.000  Deaths:  0
Day:  2020-10-19T00:00:00.000  Deaths:  41
Day:  2020-10-19T00:00:00.000  Deaths:  189
Day:  2020-10-19T00:00:00.000  Deaths:  391
Day:  2020-10-19T00:00:00.000  Deaths:  453
Day:  2020-10-19T00:00:00.000  Deaths:  390
Day:  2020-10-19T00:00:00.000  Deaths:  335
Day:  2020-10-19T00:00:00.000  Deaths:  268
Day:  2020-10-19T00:00:00.000  Deaths:  241
Day:  2020-10-19T00:00:00.000  Deaths:  170
Day:  2020-10-19T00:00:00.000  Deaths:  131
Day:  2020-10-19T00